# Download des bibliothèques

In [4]:

#TF Librairies Import 
from tensorflow.keras.metrics import TopKCategoricalAccuracy
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D, Conv2D, MaxPooling2D, AveragePooling2D, Input, GlobalMaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.optimizers.experimental import AdamW, Optimizer
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers.schedules import CosineDecayRestarts
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input as pi_16
from tensorflow.keras.applications.vgg19 import preprocess_input as pi_19
from tensorflow.keras.applications.densenet import preprocess_input as pi_densenet
from tensorflow.keras.applications.xception import preprocess_input as pi_xception
from tensorflow.keras.applications.mobilenet import preprocess_input as pi_mobilnet
from tensorflow.keras.applications import Xception, MobileNetV2, MobileNetV3Large, DenseNet201
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.optimizers import RMSprop
from tensorflow_addons.optimizers import CyclicalLearningRate
import keras

#Traditionnals librairies import
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score,
from useful_functions import create_tensorboard_callback
import datetime
from timeit import default_timer as timer
import tensorflow as tf
from PIL import Image 
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time, cv2
import seaborn as sns
import math
import glob
import absl.logging

#Display parameters
absl.logging.set_verbosity(absl.logging.ERROR)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

<ipython-input-4-1bd1c2ef0873>:54: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


# Creation des paramètres et du modèle

In [ ]:
## VARIABLE CREATION ##

#Global
top = 100
number_images_selected = 300
rank = 'species'

#Paths
origine_picture = 'gbif'
workdir = '/Users/cgm_mbf_lemery_mac/Documents/DataScience/mushroom_dataset/'
path_LR_determiner = workdir + 'LR_determiner/'
path_models = workdir + 'models/'
path_graphes = workdir + 'graphes/'
path_images = workdir +  'images/'
path_cfn = workdir +  'confusion_matrix/'
dir_logs = workdir + 'tensorboard_logs/'
path_history = workdir + 'history/'
data_images = workdir + 'data_images/'
train_directory = data_images + 'images_'+str(origine_picture)+'_'+str(top)+'_'+str(number_images_selected)+'/train/'
test_directory = data_images + 'images_'+str(origine_picture)+'_'+str(top)+'_'+str(number_images_selected)+'/test/'
validation_directory = data_images + 'images_'+str(origine_picture)+'_'+str(top)+'_'+str(number_images_selected)+'/val/'


#Callback Parameters
patience_learning_rate_tuning= 1                    #Time to wait to get learning rate adaptation callback - Tuning
patience_learning_rate_fine_tuning = 1              #Time to wait to get learning rate adaptation callback - Fine Tuning
cooldown_learning_rate_tuning =0                    #Time to wait wait after learning rate adaptation callback - Tuning
cooldown_learning_rate_fine_tuning = 0              #Time to wait wait after learning rate adaptation callback - Fine Tuning
reduce_learning_rate_type = 'ReduceLROnPlateau'     #Just for naming
factor_learning_rate_reducing_initial = 0.90        #Reduction factor for learning rate - Tuning
factor_learning_rate_reducing_tuning = 0.90         #Reduction factor for learning rate - Fine Tuning
min_delta_rlr = 0.02                                #Minimal delta value of variation to activate learning rate reduction
min_delta_stop = 0.01                               #Minimal delta value before to stop training
patience = 4                                        #Time to wait with no delta stop improvement to stop training                                    

#Models Parameters
selected_model = 'vgg19'                            #Available : VGG16, VGG19, Xception, MobilNetV2, densenet201
top_k_accuracy = 3                                  #Analyse the accuracy that the prediction is in the k first results
layer_dense = 1                                     #Number of dense layer
epoch_tuning = 45                                   #Number of epoch for the raw period
epoch_fine_tuning = 20                              #Epoch for Fine Tuning
number_phase_fine_tuning = 0                        #Si aucun tuning : 0
dropout = 0.5                                       #Dropout rate
batch_size = 80                                     #Batch size
target_size = (224,224)                             #Size of image in the batches
batch_normalization = 'Yes'                         #Is there BN in the classificator

#Lr finder parameter
start_lr = 1e-06                                    #Start Lr for LR Finder analysis
end_lr = 0.1                                        #End Lr for LR Finder analysis
epoch_lr_determiner = 20                            #Number of epoch to run the LR Finder analyse. Allow to control the number of tested value. Lr is changed every 3 batches

#Regularization
kernel_regularizer_value = 1e-02                    #Kernel regularization value in the classificator (if L1L2, then the value is L2 and L1 = L2/10)
kernel_regularizer_type = None                      #None, L1, L2, L1L2

#Optimizer
learning_rate_tuning = 3e-03
learning_rate_fine_tuning = 2e-05                   #Learning rate for Fine Tuning
weight_decay = 5e-05                                #Weight Decay                     
optimizer_type = 'RMSprop'                          #SGD, Adam et AdamW
loss = 'categorical_crossentropy'                   #Loss function


# Object creation for training

### Image Data Generator

In [ ]:
"""
This part concerns the creation of image generators. They allow to apply data augmentation in order to fight against overfitting
The index validation split allows to specify that this generator can generate two files of augmented images: One for test and one for validation
"""

selected_model = selected_model.lower()
dictionnaire_pi = {'vgg16' : pi_16,'vgg19' : pi_19, 'xception' : pi_xception, 'mobilnetv2' : pi_mobilnet, 'densenet' : pi_densenet, }

train_data_generator = ImageDataGenerator(preprocessing_function= dictionnaire_pi[selected_model],
                                          rotation_range= 45,
                                          width_shift_range= 0.22,
                                          height_shift_range = 0.22,
                                          zoom_range = 0.22,
                                          shear_range= 0.22,
                                          fill_mode='nearest',
                                          horizontal_flip=True,
                                          vertical_flip=True,
                                          #brightness_range=(0.3, 0.9), 
                                          validation_split = 0.2)
test_data_generator = ImageDataGenerator(preprocessing_function = dictionnaire_pi[selected_model])

### Image Data Iterator

In [ ]:
"""
This part concerns the creation of sets of augmented images. 
It allows you to create from the 'train' and 'test' image folders, the train, validation and test image sets
"""

train_generator = train_data_generator.flow_from_directory(directory = train_directory, 
                                                            class_mode = 'categorical', target_size = target_size, batch_size = batch_size,subset = 'training', color_mode = 'rgb', seed = 42, shuffle = True )

validation_generator = train_data_generator.flow_from_directory(directory = train_directory, 
                                                            class_mode = 'categorical', target_size = target_size, batch_size = batch_size, subset = 'validation', color_mode = 'rgb', seed = 42, shuffle = True)

test_generator = test_data_generator.flow_from_directory(directory = test_directory, 
                                                            class_mode = 'categorical', target_size = target_size, batch_size = batch_size, color_mode = 'rgb', seed = 42, shuffle = False)

# Definition des fonctions base_model Kernel_choice and add_layer

In [1]:
"""
This part concerns the creation of functions allowing the creation of models, optimizers and penalties
"""

def select_base_model(selected_model):
    """
    Base model selection
    """
    selected_model = selected_model.lower()
    if selected_model == 'vgg16':
        base_model = VGG16(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))
        return base_model
    elif selected_model == 'vgg19':
        base_model = VGG19(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))
        return base_model
    elif selected_model == 'xception':
        base_model = Xception(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3), pooling='avg')
        return base_model
    elif selected_model == 'mobilnetv2':
        base_model = MobileNetV2(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3), pooling='avg')
        return base_model
    elif selected_model == 'densenet':
        base_model = DenseNet201(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3), pooling='avg')
        return base_model
    else :
        raise ValueError('Select model : vgg16, vgg19, Xception or MobilNetV2')


def kernel_choice(kernel_regularizer_type, kernel_regularizer_value):
    """
    Penalty selection
    """
    if kernel_regularizer_type == 'l1':
        kernel_regularizer = regularizers.L1(kernel_regularizer_value)
        return kernel_regularizer
    elif kernel_regularizer_type == 'l2' :
        kernel_regularizer = regularizers.L2(kernel_regularizer_value)
        return kernel_regularizer
    elif kernel_regularizer_type == 'l1l2' :
        kernel_regularizer = regularizers.L1L2(l1 = kernel_regularizer_value, l2 = kernel_regularizer_value)
        return kernel_regularizer
    elif kernel_regularizer_type == None :
        print('Aucune regularization kernel')
    else : raise ValueError('Define kernel regularization')

def construction_model(base_model, selected_model, kernel_regularizer_type, kernel_regularizer_value):
    """
    Model construction
    """
    selected_model = selected_model.lower()
    if selected_model == 'vgg16' or selected_model == 'vgg19':
        model = Sequential()
        model.add(base_model)
        model.add(GlobalAveragePooling2D())
        model = add_layer(layer_dense, kernel_regularizer_type, model, kernel_regularizer_value)
        return model
    if selected_model == 'xception' or selected_model == 'mobilnetv2' or selected_model == 'densenet' :
        model = Sequential()
        model.add(base_model)
        model = add_layer(layer_dense, kernel_regularizer_type, model, kernel_regularizer_value)
        return model

def add_layer(layer_dense, kernel_regularizer_type, model, kernel_regularizer_value) :
    """
    Classificator construction
    """
    if kernel_regularizer_type == None :
        if layer_dense == 1:
            model.add(Dense(units = 1024, activation = 'relu'))
            if batch_normalization.lower() == 'yes':
                model.add(BatchNormalization())
            model.add(Dropout(rate =dropout))
            model.add(Dense(units = top, activation = 'softmax'))
            return model

        elif layer_dense == 2:
            model.add(Dense(units = 1024, activation = 'relu'))
            if batch_normalization.lower() == 'yes':
                model.add(BatchNormalization())
            model.add(Dropout(rate =dropout))
            model.add(Dense(units = 512, activation = 'relu'))
            if batch_normalization.lower() == 'yes':
                model.add(BatchNormalization())
            model.add(Dropout(rate = dropout))
            model.add(Dense(units = top, activation = 'softmax'))
            return model

        elif layer_dense == 3:
            model.add(Dense(units = 1024, activation = 'relu'))
            model.add(Dropout(rate =dropout))
            if batch_normalization.lower() == 'yes':
                model.add(BatchNormalization())
            model.add(Dense(units = 512, activation = 'relu'))
            model.add(Dropout(rate = dropout))
            if batch_normalization.lower() == 'yes':
                model.add(BatchNormalization())
            model.add(Dense(units = 254, activation = 'relu'))
            model.add(Dropout(rate = dropout))
            if batch_normalization.lower() == 'yes':
                model.add(BatchNormalization())
            model.add(Dense(units = top, activation = 'softmax'))
            return model
    
    elif kernel_regularizer_type == 'l1' or kernel_regularizer_type == 'l2' or kernel_regularizer_type == 'l1l2' :
        if layer_dense == 1:
            model.add(Dense(units = 1024, activation = 'relu', kernel_regularizer = kernel_choice(kernel_regularizer_type,kernel_regularizer_value)))
            model.add(Dropout(rate =dropout))
            if batch_normalization.lower() == 'yes':
                model.add(BatchNormalization())
            model.add(Dense(units = top, activation = 'softmax'))
            return model

        elif layer_dense == 2:
            model.add(Dense(units = 1024, activation = 'relu', kernel_regularizer = kernel_choice(kernel_regularizer_type, kernel_regularizer_value)))
            model.add(Dropout(rate =dropout))
            if batch_normalization.lower() == 'yes':
                model.add(BatchNormalization())
            model.add(Dense(units = 512, activation = 'relu', kernel_regularizer = kernel_choice(kernel_regularizer_type, kernel_regularizer_value)))
            model.add(Dropout(rate = dropout))
            if batch_normalization.lower() == 'yes':
                model.add(BatchNormalization())
            model.add(Dense(units = top, activation = 'softmax'))
            return model

        elif layer_dense == 3:
            model.add(Dense(units = 1024, activation = 'relu', kernel_regularizer = kernel_choice(kernel_regularizer_type, kernel_regularizer_value)))
            model.add(Dropout(rate =dropout))
            if batch_normalization.lower() == 'yes':
                model.add(BatchNormalization())
            model.add(Dense(units = 512, activation = 'relu', kernel_regularizer = kernel_choice(kernel_regularizer_type, kernel_regularizer_value)))
            model.add(Dropout(rate = dropout))
            if batch_normalization.lower() == 'yes':
                model.add(BatchNormalization())
            model.add(Dense(units = 254, activation = 'relu', kernel_regularizer = kernel_choice(kernel_regularizer_type, kernel_regularizer_value)))
            model.add(Dropout(rate = dropout))
            if batch_normalization.lower() == 'yes':
                model.add(BatchNormalization())
            model.add(Dense(units = top, activation = 'softmax'))
            return model
    else :
        raise ValueError('Issue with regularizer : is not l1, or l2, or l1l2, or None OR Define a integer between 1 and 3 for the number of dense layers')

def list_split(nnumber_phase_tuning,number_of_layer) :
    """
    List creation for Tuning / Fine-Tuning iteration
    """
    if number_phase_tuning == 0 : 
        list_splits = [0]
        return list_splits
    elif  number_phase_tuning != 0 :
        list_splits = [round((number_of_layer/number_phase_tuning)*i) for i in np.arange(0,number_phase_tuning+1)]
        return list_splits
    else :
        raise ValueError('Issue with number_splits')


def optimizer_choice(optimizer_type, learning_rate_params):
    """
    Selection des optimizers. 
    Tout nouvel optimizer doit être ajouté ici.
    Attention aux paramètres d'entrée de la fonction si de nouveaux arguments
    """
    optimizer_type = optimizer_type.lower()
    if optimizer_type == 'adam':
        optimizer = Adam(learning_rate = learning_rate_params)
        return optimizer
    elif optimizer_type == 'adamw':
        optimizer = AdamW(learning_rate= learning_rate_params,  beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, clipnorm=None,
        clipvalue=None, global_clipnorm=None, use_ema=False, ema_momentum=0.99, ema_overwrite_frequency=None, jit_compile=False, name="AdamW")
        return optimizer
    elif optimizer_type == 'adamcdr':
        lr_decayed = CosineDecayRestarts(initial_learning_rate = learning_rate_params, weight_decay = weight_decay, alpha = 0.01)
        optimizer = Adam(learning_rate= lr_decayed)
        return optimizer
    elif optimizer_type == 'sgd':
        optimizer = SGD(learning_rate =  learning_rate_params, weight_decay = weight_decay)
        return optimizer
    elif optimizer_type == 'rmsprop':
        optimizer = RMSprop(learning_rate =  learning_rate_params)
        return optimizer
    else : raise ValueError('Optimizer non enregistré')

#Learning rate reduction callback
def step_decay(epoch, lr):
    drop = 0.98
    epochs_drop = 1
    return  lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))


# Model creation

## Select just one of the two following method : Sequential or Functionnal. Functionnal is not automatic

In [ ]:
### SEQUENTIAL MODEL CREATION ##

#Model de base et model total
base_model = select_base_model(selected_model)
model = construction_model(base_model, selected_model, kernel_regularizer_type, kernel_regularizer_value)

#Model de base et model total
number_of_layer = len(base_model.layers)
list_splits =list_split(number_phase_fine_tuning,number_of_layer)

In [ ]:
### FONCTIONNAL MODEL CREATION ##

dense_1_units = 1024
dense_2_units = 512
dense_3_units = 256

#Model creation
dense1 = Dense(units = dense_1_units, activation = 'relu', kernel_initializer = 'normal', name = 'Dense_1')
dense2 = Dense(units = dense_2_units, activation = 'relu', kernel_initializer = 'normal', name = 'Dense_2')
dense3 = Dense(units = dense_3_units, activation = 'relu', kernel_initializer = 'normal', name = 'Dense_3')

max_pooling = GlobalMaxPooling2D(data_format=None, keepdims=False)
avg_pooling = GlobalAveragePooling2D(data_format=None, keepdims=False)
dense_softmax = Dense(units = top, activation = 'softmax', kernel_initializer = 'normal', name = 'Dense_softmax')
dropout_layer = Dropout(dropout, noise_shape=None, seed=None)
bn_layer = BatchNormalization()


inputs = Input(shape=(224, 224, 3))
base_model = DenseNet201(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))
x = base_model(inputs)
x2_prim = avg_pooling(x)
x2_bis = max_pooling(x)
x2 = tf.keras.layers.concatenate([x2_prim, x2_bis])
x3 = dense1(x2)
x4 = dropout_layer(x3)
x5 = bn_layer(x4)
x6 = dense_softmax(x5)
Outputs = x6
model = Model(inputs = inputs, outputs = Outputs)

#Split list creation
number_of_layer = len(base_model.layers)
list_splits =list_split(number_phase_fine_tuning,number_of_layer)

### Training loop

In [ ]:
#Naming and folder creation
name_history = (str(selected_model)+'_'+str(top)+'_'+str(number_images_selected)+'_'+str(rank)+str(reduce_learning_rate_type)+'_'+'LR_raw-'+str("%.1e" % learning_rate_raw_initial)+'_pat_lr_init-'+str(patience_learning_rate_initial)+'_coold_lr_init-'+str(cooldown_learning_rate_initial)+'_deltaRLR-'+str(min_delta_rlr)+'_optim-'+str(optimizer_type)+'_epochs_init-'+str(epoch_raw)+'_'+'DO-'+str(dropout)+'_'+'layer_dense-'+str(layer_dense)+'_'+'kernel_reg-'+str(kernel_regularizer_type)+'-'+str(kernel_regularizer_value)+'_'+origine_picture+'_'+'BatchNorm-'+str(batch_normalization))
directory_name = str(datetime.datetime.now()).replace(":", '_').replace(" ", '_')
path_history_deep = os.path.join(path_history, directory_name + '_' + name_history)
path_graphes_deep = os.path.join(path_graphes, directory_name + '_' + name_history)
path_models_deep = os.path.join(path_models, directory_name + '_' + name_history)
os.mkdir(path_history_deep)
os.mkdir(path_graphes_deep)
os.mkdir(path_models_deep)

#List creation for Tuning and Fine-Tuning iterations
name_params = []
path_model_params = []
path_graphs_params = []
learning_rate_params = []
factor_learning_rate_reducing = []
patience_learning_rate_params = []
cooldown_learning_rate_params = []
epochs =[]

#Dataframe for history concatenation
df = pd.DataFrame()

#Model training loop: We go through the list of plit to train the classifier, then the CNN layers by tuning
#The parameters are identical between the tuning phases, except for the learning rate

for index, split in enumerate(list_splits) :

    #Définition of variables for loops
    if index == 0: #RAW
        learning_rate_params.append(learning_rate_tuning)
        factor_learning_rate_reducing.append(factor_learning_rate_reducing_initial)
        patience_learning_rate_params.append(patience_learning_rate_tuning)
        cooldown_learning_rate_params.append(cooldown_learning_rate_tuning)
        epochs.append(epoch_tuning)
    else :  #TUNING
        learning_rate_params.append(learning_rate_fine_tuning/(1.2**(index)))
        factor_learning_rate_reducing.append(factor_learning_rate_reducing_tuning)
        patience_learning_rate_params.append(patience_learning_rate_fine_tuning)
        cooldown_learning_rate_params.append(cooldown_learning_rate_fine_tuning)
        epochs.append(epoch_fine_tuning)


    #Définition des noms pour les enregistrement (on sauvegarde les fichiers par phases)
    name_params.append(str(top)+'_'+str(number_images_selected)+'_'+str(rank)+'_split_num-'+str(index)+'_RLR_policy-'+str(reduce_learning_rate_type)+'_'+'flr_reduc_ini-'+str("%.1e" % learning_rate_params[index])+'_pat_lr-'+str(patience_learning_rate_params[index])+'_coold_lr_'+str(cooldown_learning_rate_params[index])+'_deltaRLR-'+str(min_delta_rlr)+'_optim-'+str(optimizer_type)+'_epochs-'+str(epochs[index])+'_'+'DO-'+str(dropout)+'_'+'layer_dense-'+str(layer_dense)+'_'+'kernel_reg-'+str(kernel_regularizer_type)+'-'+str(kernel_regularizer_value)+'_'+origine_picture+'_BatchNorm-'+str(batch_normalization))
    path_model_params.append(path_models_deep+'/'+(selected_model)+'_'+str(top)+'_'+str(number_images_selected)+'_'+str(rank)+'_split_num-'+str(index)+'_RLR_policy-'+str(reduce_learning_rate_type)+'_flr_reduc_init-'+str("%.1e" %  learning_rate_params[index])+'_pat_lr-'+str(patience_learning_rate_params[index])+'_coold_lr_'+str(cooldown_learning_rate_params[index])+'_deltaRLR-'+str(min_delta_rlr)+'_optim-'+str(optimizer_type)+'_epochs-'+str(epochs[index])+'_'+'DO-'+str(dropout)+'layer_dense-'+str(layer_dense)+'_'+'kernel_reg-'+str(kernel_regularizer_type)+'-'+str(kernel_regularizer_value)+'_BatchNorm-'+str(batch_normalization)+'_'+origine_picture+'.h5')
    path_graphs_params.append(path_graphes_deep+'/'+(selected_model)+'_'+str(top)+'_'+str(number_images_selected)+'_'+str(rank)+'_split_num-'+str(index)+'_RLR_policy-'+str(reduce_learning_rate_type)+'_flr_reduc_init-'+str("%.1e" % learning_rate_params[index])+'_pat_lr-'+str(patience_learning_rate_params[index])+'_coold_lr_'+str(cooldown_learning_rate_params[index])+'_deltaRLR-'+str(min_delta_rlr)+'_optim-'+str(optimizer_type)+'_epochs-'+str(epochs[index])+'_'+'DO-'+str(dropout)+'_'+'layer_dense-'+str(layer_dense)+'_'+'kernel_reg-'+str(kernel_regularizer_type)+'-'+str(kernel_regularizer_value)+'_BatchNorm-'+str(batch_normalization)+'_'+origine_picture+'.pdf')
  
    #Optimizer
    print(learning_rate_params[index])
    optimizer = optimizer_choice(optimizer_type, learning_rate_params[index])

    #Callbacks
    reduce_learning_rate = ReduceLROnPlateau(monitor = 'val_loss', factor = factor_learning_rate_reducing[index], patience = patience_learning_rate_params[index], cooldown = cooldown_learning_rate_params[index], verbose = 1, min_delta=min_delta_rlr)
    earlystop = EarlyStopping(monitor = 'val_loss', min_delta = min_delta_stop, patience = patience, verbose = 1, restore_best_weights = True)
    LRscheduler = LearningRateScheduler(step_decay)
    model_check = ModelCheckpoint(filepath = path_model_params[index], monitor = 'val_loss', mode = 'min')
    tensorboard = create_tensorboard_callback(dir_logs, name_params[index])
       
    if index == 0: #Tuning
       #Trainable layers
        for layer in base_model.layers:
            layer.trainable = False

    else : #Fine-tuning
        for layer in base_model.layers[-split:]:
            layer.trainable = True 


    #Model training
    print('Iteration : {}'.format(index), '\n')
    print(model.summary())
    model.compile(optimizer = optimizer, loss = loss, metrics = ['acc', TopKCategoricalAccuracy(top_k_accuracy)])
    history = model.fit(train_generator, epochs = epochs[index], verbose = 1, validation_data = validation_generator, 
                        callbacks = [model_check, reduce_learning_rate, tensorboard, earlystop, LRscheduler])
    
    #History backup
    pd.DataFrame.from_dict(history.history).to_csv(path_history_deep + '/' + name_params[index] ,index=False)

    df = pd.concat([df, pd.DataFrame.from_dict(history.history)], axis = 0)
    df.to_csv(path_history_deep + '/' + name_history ,index=False)
    

    #Plots
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    train_acc = history.history["acc"]
    val_acc = history.history["val_acc"]
    fig = plt.figure(figsize = (20, 12))
    fig = plt.subplot(211)
    fig = plt.plot(train_loss)
    fig = plt.plot(val_loss)
    fig = plt.title('Model Cross_entropy : ' + name_params[index])
    fig = plt.ylabel('loss')
    fig = plt.xlabel('epoch')
    fig = plt.legend(['train', 'validation'], loc='right')
    fig = plt.subplot(212)
    fig = plt.plot(train_acc)
    fig = plt.plot(val_acc)
    fig = plt.title('Model accuracy per epoch (ACC) : ' + name_params[index])
    fig = plt.ylabel('acc')
    fig = plt.xlabel('epoch')
    fig = plt.legend(['train', 'validation'], loc='right')
    plt.savefig(path_graphs_params[index])
    fig = plt.show()

    #Accuracy on testing data
    test_pred = model.predict(test_generator)
    test_pred_class = test_pred.argmax(axis = 1)
    y_test = test_generator.classes
    y_test_one_hot = pd.get_dummies(y_test)
    m = tf.keras.metrics.TopKCategoricalAccuracy(k=top_k_accuracy)
    m.update_state(y_test_one_hot, test_pred)
    result_k = m.result().numpy()
    print('The k_accuracy score iteration {} : '.format(index),result_k)
    score = accuracy_score(y_test, test_pred_class)
    print('The accuracy score iteration {} : '.format(index), score)

# LR Finder Analysis

In [ ]:
"""
Class to calcul the best Lr according to Smith method. Modified from Github.
"""

from keras.callbacks import Callback
import keras.backend as K
import numpy as np
import matplotlib.pyplot as plt

class LRFinder(Callback):
    def __init__(self, min_lr, max_lr, path, name, mom=0.9, stop_multiplier=None, 
                 reload_weights=True, batches_lr_update=2):
        self.min_lr = min_lr
        self.max_lr = max_lr
        self.name = name
        self.mom = mom
        self.path = path
        self.reload_weights = reload_weights
        self.batches_lr_update = batches_lr_update
        if stop_multiplier is None:
            self.stop_multiplier = -20*self.mom/3 + 10 # 4 if mom=0.9
                                                       # 10 if mom=0
        else:
            self.stop_multiplier = stop_multiplier
        
    def on_train_begin(self, logs={}):
        p = self.params
        print(p)
        try:
            n_iterations = p['epochs']*p['samples']//p['batch_size']
        except:
            n_iterations = p['steps']*p['epochs']
            
        self.learning_rates = np.geomspace(self.min_lr, self.max_lr, \
                                           num=n_iterations//self.batches_lr_update+1)
        
        self.losses=[]
        self.iteration=0
        self.best_loss=0
        if self.reload_weights:
            self.model.save_weights('tmp.hdf5')

    
    def on_batch_end(self, batch, logs={}):
        loss = logs.get('loss')
        
        if self.iteration==0 or loss < self.best_loss: 
            self.best_loss = loss


        if self.iteration!=0: # Make loss smoother using momentum
            loss = logs.get('loss')
            loss = self.losses[-1]*self.mom+loss*(1-self.mom)
                
        if self.iteration%self.batches_lr_update==0 :
            loss = logs.get('loss')

            if self.reload_weights:
                self.model.load_weights('tmp.hdf5')
          
            lr = self.learning_rates[self.iteration//self.batches_lr_update]            
            K.set_value(self.model.optimizer.lr, lr)
            
            self.losses.append(loss)


        if loss > self.best_loss*self.stop_multiplier: # Stop criteria
            self.model.stop_training = True
            self.losses.append(loss)

        self.iteration += 1
    
    def on_train_end(self, logs=None):
        if self.reload_weights:
                self.model.load_weights('tmp.hdf5')
                
        plt.figure(figsize=(12, 6))
        plt.plot(self.learning_rates[:len(self.losses)], self.losses[:])
        plt.xlabel("Learning Rate")
        plt.ylabel("Loss")
        plt.xscale('log')
        plt.show()
        df_temp = pd.DataFrame({'Learning_rate' :self.learning_rates[:len(self.losses)], 'Losse' :self.losses[0:]})
        df_temp.to_csv(self.path + '/' + self.name ,index=False)

In [ ]:
#Naming and folder creation
name_history = ('LR_DETERMINATION-'+str(selected_model)+'_'+str(top)+'_'+str(number_images_selected)+'_'+str(rank)+'_optim-'+str(optimizer_type)+'_'+'DO-'+str(dropout)+'_'+'layer_dense-'+str(layer_dense)+'_'+origine_picture)
directory_name = str(datetime.datetime.now()).replace(":", '_').replace(" ", '_')
path_lr_determiner_deep = os.path.join(path_LR_determiner, directory_name + '_' + name_history)
os.mkdir(path_lr_determiner_deep)

In [ ]:
#Lr finder training
model.compile(loss = loss,metrics=['accuracy'], optimizer=optimizer_type)
lr_finder = LRFinder(min_lr=start_lr, max_lr= end_lr, path = path_lr_determiner_deep, name= name_history)
model.fit(train_generator, batch_size=50, callbacks=[lr_finder], epochs= epoch_lr_determiner, validation_data = validation_generator)